In [81]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy
from datetime import datetime
import statsmodels.formula.api as smf

from matplotlib import style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
from pylab import mpl
import platform

# 根据操作系统设置中文字体
system = platform.system()
if system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows使用黑体
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
elif system == 'Darwin':  # macOS
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # macOS使用Arial Unicode MS
    plt.rcParams['axes.unicode_minus'] = False
else:  # Linux
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei']  # Linux使用文泉驿微米黑
    plt.rcParams['axes.unicode_minus'] = False

# 输出矢量图
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 设置pandas显示选项
pd.set_option('display.max_columns', None)

print(f"当前操作系统: {system}")
print(f"字体设置: {plt.rcParams['font.sans-serif']}")

当前操作系统: Darwin
字体设置: ['Arial Unicode MS']


# 核心解释变量：人工智能投资

In [82]:
# 导入数据
file_path = 'Data/CNDD-0299 上市公司人工智能投资水平/CNDD-0299 上市公司人工智能投资水平.xls'
df = pd.read_excel(file_path)

# 补齐6位股票代码
df['证券代码'] = df['证券代码'].astype(str).str.zfill(6)

# rename columns for easier access
df.rename(columns={'证券代码': 'Stkcd',
                   '人工智能软件投资（元）': 'AI_Invest_soft',
                   '人工智能硬件投资（元）': 'AI_Invest_hard',
                   '人工智能总投资（元）': 'AI_Invest',
                   '人工智能投资水平': 'AI_Invest_Level',
                   '统计年度': 'Year'},
          inplace=True)

# 计算总投资
df['Total_Invest'] = df['AI_Invest'] / df['AI_Invest_Level']
df['AI_Invest_Level_Soft'] = df['AI_Invest_soft'] / df['Total_Invest']
df['AI_Invest_Level_Hard'] = df['AI_Invest_hard'] / df['Total_Invest']
df.drop(columns=['证券简称'], inplace=True)
# 查看数据前几行
print("数据前5行:")
df

数据前5行:


,Stkcd,AI_Invest_soft,AI_Invest_hard,AI_Invest,AI_Invest_Level,Year,Total_Invest,AI_Invest_Level_Soft,AI_Invest_Level_Hard
0,000001,3.758920e+08,1.096640e+09,1.472532e+09,0.0012,2011,1.227110e+12,0.000306,0.000894
1,000001,5.527070e+08,1.194219e+09,1.746926e+09,0.0011,2012,1.588115e+12,0.000348,0.000752
2,000001,4.230000e+08,1.388000e+09,1.811000e+09,0.0010,2013,1.811000e+12,0.000234,0.000766
3,000001,5.390000e+08,1.656000e+09,2.195000e+09,0.0010,2014,2.195000e+12,0.000246,0.000754
4,000001,4.960000e+08,1.882000e+09,2.378000e+09,0.0009,2015,2.642222e+12,0.000188,0.000712
...,...,...,...,...,...,...,...,...,...
47828,920167,1.038300e+06,NaN,1.038300e+06,0.0005,2024,2.076600e+09,0.000500,NaN
47829,920445,5.236678e+05,NaN,5.236678e+05,0.0010,2024,5.236678e+08,0.001000,NaN
47830,920682,1.686236e+06,NaN,1.686236e+06,0.0007,2024,2.408908e+09,0.000700,NaN
47831,920799,1.007692e+07,NaN,1.007692e+07,0.0088,2024,1.145104e+09,0.008800,NaN


# 核心被解释变量


## 崩盘风险

In [83]:
# 股价崩盘风险指标导入与处理
crash_path = 'Data/股价崩盘指标表(年)/BF_CRASHRISK.csv'

# 读取 CSV（尝试多种编码以防中文路径或内容编码差异）
_encodings = ['utf-8', 'gb18030', 'latin1']
for enc in _encodings:
    try:
        df_crash = pd.read_csv(crash_path, encoding=enc)
        break
    except Exception as e:
        if enc == _encodings[-1]:
            raise e
        else:
            print(f'编码 {enc} 失败，尝试下一种编码...')

# 基本规范化与重命名
rename_map = {
    'Stkcd': 'Stkcd',
    'Trdynt': 'Year'
}
# 确保股票代码为字符串并补齐6位
if 'Stkcd' in df_crash.columns:
    df_crash['Stkcd'] = df_crash['Stkcd'].astype(str).str.zfill(6)

# 年份转为整数
if 'Trdynt' in df_crash.columns:
    df_crash['Year'] = pd.to_numeric(df_crash['Trdynt'], errors='coerce').astype('Int64')

# NCSKEW_Cmdtl [NCSKEW(综合市场总市值平均法)] - 字段说明见说明书“股价崩盘指标”。
# DUVOL_Cmdtl [DUVOL(综合市场总市值平均法)] - 字段说明见说明书“股价崩盘指标”。
df_crash.rename(columns={
    'NCSKEW_Cmdtl': 'NCSKEW',
    'DUVOL_Cmdtl': 'DUVOL'
}, inplace=True)

df_crash = df_crash[['Stkcd', 'Year', 'NCSKEW', 'DUVOL']]
df_crash

,Stkcd,Year,NCSKEW,DUVOL
0,000001,1991,1.332329,1.202898
1,000001,1992,0.384722,0.055806
2,000001,1993,0.499522,0.458513
3,000001,1994,-2.054718,-0.987508
4,000001,1995,-0.604716,-0.766091
...,...,...,...,...
68753,920982,2024,-0.147918,0.149732
68754,920985,2023,-0.673540,-0.785112
68755,920985,2024,-3.257187,-1.121180
68756,920992,2023,-1.932683,-1.668855


## 贝塔

In [ ]:
# # 读取 Beta（RDS）数据
# beta_path = 'Data/Beta2024.RDS'

# # 优先使用 pyreadr，若无则安装;若失败再尝试 rpy2
# _beta_df = None
# try:
#     import pyreadr
# except ImportError:
#     try:
#         %pip install -q pyreadr
#         import pyreadr
#     except Exception as e:
#         print('pyreadr 安装失败:', e)
#         pyreadr = None

# if pyreadr is not None:
#     try:
#         result = pyreadr.read_r(beta_path)
#         # pyreadr 返回字典：键为对象名；通常 RDS 只有一个对象
#         if len(result) == 1:
#             _beta_df = list(result.values())[0]
#         else:
#             # 多对象时尝试寻找包含 Beta 或类似命名的
#             for k, v in result.items():
#                 if 'beta' in k.lower():
#                     _beta_df = v
#                     break
#             if _beta_df is None:
#                 _beta_df = list(result.values())[0]
#     except Exception as e:
#         print('pyreadr 读取失败，尝试 rpy2:', e)

# # 如果 pyreadr 失败，尝试 rpy2
# if _beta_df is None:
#     try:
#         import rpy2.robjects as ro
#         from rpy2.robjects import pandas2ri
#         pandas2ri.activate()
#         beta_r = ro.r["readRDS"](beta_path)
#         _beta_df = pandas2ri.rpy2py(beta_r)
#     except Exception as e:
#         print('rpy2 读取失败，无法加载 Beta 数据:', e)

# if _beta_df is None:
#     raise FileNotFoundError('无法成功读取 Beta RDS 文件，请确认路径与文件格式。')

# # 标准化列名与关键字段
# beta_df = _beta_df.copy()
# # 统一列名为小写去除空格
# beta_df.columns = [c.strip() for c in beta_df.columns]

# # 将 month 列转换为日期格式 (YYYY-MM-DD)
# # month 格式为 year.fraction，其中 fraction 表示月份
# # 例如: 1992.0833333 = 2月, 1992.1666667 = 3月, 1992.25 = 4月
# # 规律：month = round(fraction * 12) + 1
# def convert_month_to_date(month_decimal):
#     year = int(month_decimal)
#     fraction = month_decimal - year
#     month = int(round(fraction * 12)) + 1
    
#     # 处理边界情况
#     if month > 12:
#         month = 1
#         year += 1
#     elif month < 1:
#         month = 12
#         year -= 1
    
#     # 使用该月最后一天
#     return pd.Period(year=year, month=month, freq='M').to_timestamp('M')

# beta_df['month'] = beta_df['month'].apply(convert_month_to_date)

# beta_df = beta_df[['Stkcd', 'month', 'beta_12m','beta_sd_12m','N_12m']]

# # 去每年的最后一个观测值
# beta_df['Year'] = beta_df['month'].dt.year
# beta_df_yearly = beta_df.sort_values(['Stkcd', 'month']).groupby(['Stkcd', 'Year']).last().reset_index()
# beta_df_yearly

,Stkcd,Year,month,beta_12m,beta_sd_12m,N_12m
0,000001,1991,1991-12-31,1.320747,0.046530,198
1,000001,1992,1992-12-31,0.547526,0.039650,256
2,000001,1993,1993-12-31,0.539055,0.033224,259
3,000001,1994,1994-12-31,0.648856,0.028817,249
4,000001,1995,1995-12-31,0.726016,0.025949,243
...,...,...,...,...,...,...
67141,605598,2024,2024-12-31,1.297851,0.123806,242
67142,605599,2021,2021-12-31,0.906701,0.450657,74
67143,605599,2022,2022-12-31,0.710316,0.100772,242
67144,605599,2023,2023-12-31,0.437232,0.162883,242


In [ ]:
# beta_df_yearly.to_csv('Data/Beta_Yearly.csv', index=False)

In [88]:
beta_df_yearly = pd.read_csv('Data/Beta_Yearly.csv')
beta_df_yearly['Stkcd'] = beta_df_yearly['Stkcd'].astype(str).str.zfill(6)
beta_df_yearly

,Stkcd,Year,month,beta_12m,beta_sd_12m,N_12m
0,000001,1991,1991-12-31,1.320747,0.046530,198.0
1,000001,1992,1992-12-31,0.547526,0.039650,256.0
2,000001,1993,1993-12-31,0.539055,0.033224,259.0
3,000001,1994,1994-12-31,0.648856,0.028817,249.0
4,000001,1995,1995-12-31,0.726016,0.025949,243.0
...,...,...,...,...,...,...
67141,605598,2024,2024-12-31,1.297851,0.123806,242.0
67142,605599,2021,2021-12-31,0.906701,0.450657,74.0
67143,605599,2022,2022-12-31,0.710316,0.100772,242.0
67144,605599,2023,2023-12-31,0.437232,0.162883,242.0


# 公司性质

In [ ]:
firms = pd.read_excel('Data/TRD_Co.xlsx')
firms = firms[['Stkcd','Indcd']]
# 删除前两行
firms = firms.iloc[2:].copy()
firms

# Indcd 0001是金融企业，需要删除

/opt/anaconda3/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Stkcd,Indcd
2,000001,0001
3,000002,0003
4,000003,0004
5,000004,0002
6,000005,0002
...,...,...
5787,920445,0005
5788,920489,0005
5789,920682,0005
5790,920799,0002


# 控制变量

# 固定效应回归

In [108]:
# 合并数据
df_merged = df.merge(df_crash, on=['Stkcd', 'Year'], how='inner')
df_merged = pd.merge(df_merged,beta_df_yearly, on=['Stkcd', 'Year'], how='inner')
df_merged = df_merged.merge(firms, on='Stkcd', how='left')
df_merged = df_merged[df_merged['Indcd'] != '0001']  # 删除金融企业
df_merged

,Stkcd,AI_Invest_soft,AI_Invest_hard,AI_Invest,AI_Invest_Level,Year,Total_Invest,AI_Invest_Level_Soft,AI_Invest_Level_Hard,NCSKEW,DUVOL,month,beta_12m,beta_sd_12m,N_12m,Indcd
14,000002,NaN,5.901127e+07,5.901127e+07,0.0003,2010,1.967042e+11,NaN,0.0003,-0.463988,-0.319690,2010-12-31,0.961293,0.075641,240.0,0003
15,000002,NaN,6.757055e+07,6.757055e+07,0.0002,2011,3.378528e+11,NaN,0.0002,0.074064,-0.075240,2011-12-31,0.960729,0.066734,242.0,0003
16,000002,NaN,7.217291e+07,7.217291e+07,0.0002,2012,3.608646e+11,NaN,0.0002,-0.424873,-0.203201,2012-12-31,0.978782,0.075977,238.0,0003
17,000002,NaN,8.558682e+07,8.558682e+07,0.0002,2013,4.279341e+11,NaN,0.0002,-1.472395,-0.764004,2013-12-31,1.298813,0.106123,227.0,0003
18,000002,NaN,1.069330e+08,1.069330e+08,0.0002,2014,5.346649e+11,NaN,0.0002,-0.999416,-1.047874,2014-12-31,1.122525,0.109627,245.0,0003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41679,605598,367743.60,NaN,3.677436e+05,0.0002,2023,1.838718e+09,0.0002,NaN,-0.787794,-0.430037,2023-12-31,1.557756,0.286994,242.0,0003
41680,605598,317655.06,NaN,3.176551e+05,0.0001,2024,3.176551e+09,0.0001,NaN,0.410116,0.176025,2024-12-31,1.297851,0.123806,242.0,0003
41681,605599,22777254.10,NaN,2.277725e+07,0.0042,2022,5.423156e+09,0.0042,NaN,0.219845,0.059397,2022-12-31,0.710316,0.100772,242.0,0006
41682,605599,22876291.13,NaN,2.287629e+07,0.0036,2023,6.354525e+09,0.0036,NaN,-0.400661,-0.467680,2023-12-31,0.437232,0.162883,242.0,0006


In [109]:
df_merged.columns

Index(['Stkcd', 'AI_Invest_soft', 'AI_Invest_hard', 'AI_Invest',
       'AI_Invest_Level', 'Year', 'Total_Invest', 'AI_Invest_Level_Soft',
       'AI_Invest_Level_Hard', 'NCSKEW', 'DUVOL', 'month', 'beta_12m',
       'beta_sd_12m', 'N_12m', 'Indcd'],
      dtype='object')

In [114]:
# 固定效应回归

# 准备面板数据
df_panel = df_merged.copy()

# 对连续变量进行缩尾处理 (1% 和 99% 分位数)
from scipy.stats.mstats import winsorize

# 需要缩尾的变量列表
vars_to_winsorize = ['beta_12m', 'NCSKEW', 'DUVOL', 'AI_Invest_Level']

for var in vars_to_winsorize:
    if var in df_panel.columns:
        # 删除缺失值后进行缩尾
        non_missing = df_panel[var].notna()
        if non_missing.sum() > 0:
            df_panel.loc[non_missing, var] = winsorize(
                df_panel.loc[non_missing, var], 
                limits=[0.01, 0.01]  # 在1%和99%分位数进行缩尾
            )

# 创建滞后一期的解释变量
# 按公司排序，然后对AI投资水平进行滞后
df_panel = df_panel.sort_values(['Stkcd', 'Year'])
df_panel['AI_Invest_Level_lag1'] = df_panel.groupby('Stkcd')['AI_Invest_Level'].shift(1)

print("\n滞后变量创建完成")
print(f"原变量缺失值: {df_panel['AI_Invest_Level'].isna().sum()}")
print(f"滞后变量缺失值: {df_panel['AI_Invest_Level_lag1'].isna().sum()}")

# 设置多重索引 (个体, 时间)
df_panel = df_panel.set_index(['Stkcd', 'Year'])

# 删除缺失值 - 使用滞后一期的解释变量
df_panel_reg = df_panel[['DUVOL', 'AI_Invest_Level_lag1']].dropna()

print(f"\n回归样本量: {len(df_panel_reg)}")
print(f"公司数量: {df_panel_reg.index.get_level_values(0).nunique()}")
print(f"年份跨度: {df_panel_reg.index.get_level_values(1).min()} - {df_panel_reg.index.get_level_values(1).max()}")

# 使用 linearmodels 进行固定效应回归
from linearmodels.panel import PanelOLS
    
# 双向固定效应回归 (个体固定效应 + 时间固定效应)
# 使用滞后一期的AI投资水平
model = PanelOLS.from_formula('DUVOL ~ AI_Invest_Level_lag1 + EntityEffects + TimeEffects', 
                               data=df_panel_reg)
    
# 使用稳健标准误 (clustered by entity)
result = model.fit(cov_type='clustered', cluster_entity=True)
    
print("\n" + "="*80)
print("固定效应回归结果 (双向固定效应, 解释变量滞后一期)")
print("="*80)
print(result)


滞后变量创建完成
原变量缺失值: 0
滞后变量缺失值: 4484

回归样本量: 35946
公司数量: 4299
年份跨度: 2011 - 2024

固定效应回归结果 (双向固定效应, 解释变量滞后一期)
                          PanelOLS Estimation Summary                           
Dep. Variable:                  DUVOL   R-squared:                     3.763e-05
Estimator:                   PanelOLS   R-squared (Between):             -0.0107
No. Observations:               35946   R-squared (Within):            2.302e-05
Date:                Sat, Nov 22 2025   R-squared (Overall):             -0.0039
Time:                        17:44:48   Log-likelihood                -2.091e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.1905
Entities:                        4299   P-value                           0.2752
Avg Obs:                       8.3615   Distribution:                 F(1,31633)
Min Obs:                       1.0000                                           
Max